In [4]:
import pandas as pd
from sqlalchemy import create_engine, inspect, text
import os

In [8]:
engine = create_engine(os.environ["REDSHIFT_READ_URI"], echo=False)

def sql_df(query):
  with engine.connect() as conexao:
    consulta = conexao.execute(text(query))
    dados = consulta.fetchall()
  df = pd.DataFrame(dados, columns= consulta.keys())
  return df

In [10]:
query = '''
select 
*
from consolidacao_carteira.contact_37 c
where has_hot is true and contact_type = 'CELLPHONE'
'''

df = sql_df(query)

In [9]:
query2 = '''
select 
*
from consolidacao_carteira.contact_37 c
where contact_type = 'EMAIL'
'''

df_email = sql_df(query2)

In [11]:
cell_hot =  df.groupby('customer_id').agg('max','contact_final_score').reset_index()

In [12]:
df_final = cell_hot[~cell_hot['customer_id'].isin(df_email['customer_id'])].reset_index().sort_values('notify_count', ascending = False)
df_final = df_final[['customer_id','contact_id','notify_count']]

In [14]:
df_final.to_csv('hot_sem_email.csv')